# TOPIC: Data Warehousing Fundamentals
   1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.
   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.
   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.


In [ ]:
-- Create the fact table
CREATE TABLE sales (
    product_id INT,
    customer_id INT,
    date_id INT,
    sales_amount DECIMAL(10, 2),
    units_sold INT,
    PRIMARY KEY (product_id, customer_id, date_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (date_id) REFERENCES time(date_id)
);

-- Populate the fact table with sample data
INSERT INTO sales (product_id, customer_id, date_id, sales_amount, units_sold)
VALUES
    (1, 101, 1, 100.00, 5),
    (2, 102, 1, 50.00, 2),
    (1, 103, 2, 75.00, 3),
    (3, 101, 2, 200.00, 8),
    ...
SELECT t.year, SUM(s.sales_amount) AS total_sales_amount
FROM sales s
JOIN time t ON s.date_id = t.date_id
GROUP BY t.year;

SELECT p.category, t.quarter, SUM(s.sales_amount) AS total_sales_amount
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN time t ON s.date_id = t.date_id
WHERE p.category = 'Electronics' AND t.quarter = 3
GROUP BY p.category, t.quarter;

SELECT c.city, p.brand, SUM(s.sales_amount) AS total_sales_amount
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
JOIN products p ON s.product_id = p.product_id
GROUP BY c.city, p.brand;


# TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.


In [ ]:
import csv
import psycopg2  # Assuming PostgreSQL data warehouse

# Extraction: Read data from CSV file
def extract_data(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        return [row for row in reader]

# Transformation: Apply business rules and calculations
def transform_data(data):
    transformed_data = []
    for row in data:
        # Apply business rules and calculations
        transformed_row = {
            'column1': row['column1'],
            'column2': int(row['column2']),
            'column3': float(row['column3']),
            'column4': row['column4'].upper(),
            # Additional transformations and business rules...
        }
        transformed_data.append(transformed_row)
    return transformed_data

# Loading: Load transformed data into data warehouse
def load_data(data):
    conn = psycopg2.connect(
        host="your_host",
        port="your_port",
        database="your_database",
        user="your_user",
        password="your_password"
    )
    cursor = conn.cursor()

    # Create or update tables in the data warehouse
    create_table_query = """
        CREATE TABLE IF NOT EXISTS my_table (
            column1 TEXT,
            column2 INT,
            column3 FLOAT,
            column4 TEXT
            -- Additional columns...
        )
    """
    cursor.execute(create_table_query)

    # Load transformed data into the data warehouse
    insert_query = """
        INSERT INTO my_table (column1, column2, column3, column4)
        VALUES (%s, %s, %s, %s)
    """
    for row in data:
        cursor.execute(insert_query, (row['column1'], row['column2'], row['column3'], row['column4']))

    conn.commit()
    cursor.close()
    conn.close()

# Main ETL Process
def etl_process():
    # Extract data from CSV files
    extracted_data = extract_data('data.csv')

    # Transform data
    transformed_data = transform_data(extracted_data)

    # Load transformed data into the data warehouse
    load_data(transformed_data)

# Run the ETL process
etl_process()


# TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables


In [ ]:
SELECT t.year, c.department, COUNT(*) AS enrollment_count
FROM 
enrollments e
JOIN time t ON e.date_id = t.date_id
JOIN courses c ON e.course_id = c.course_id
GROUP BY t.year, c.department;


SELECT e.enrollment_id, s.student_name, c.course_name, t.date
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id
JOIN time t ON e.date_id = t.date_id
WHERE e.enrollment_id = 12345;


# TOPIC: Performance Optimization and Querying
    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:
Utilize batch processing techniques to load data in bulk instead of individual row insertion.
      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.


In [ ]:
import time
from multiprocessing import Pool
from pymongo import MongoClient

# Function to load data in bulk using batch processing
def load_data_in_bulk(data):
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['mycollection']

    # Bulk insert the data
    collection.insert_many(data)

    client.close()

# Function to load data in parallel using multiprocessing
def load_data_parallel(data):
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['mycollection']

    # Define the number of processes (can be adjusted based on system resources)
    num_processes = 4

    # Split data into chunks
    chunk_size = len(data) // num_processes
    data_chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

    # Create a pool of worker processes
    with Pool(processes=num_processes) as pool:
        # Parallelize the data loading process
        pool.map(collection.insert_many, data_chunks)

    client.close()

# Main script
def main():
    # Generate sample data to load into the data warehouse
    data = [
        {"id": 1, "name": "John Doe"},
        {"id": 2, "name": "Jane Smith"},
        {"id": 3, "name": "Alice Johnson"},
        # Add more sample data...
    ]

    # Measure time taken to load data before optimizations
    start_time = time.time()
    load_data_in_bulk(data)
    end_time = time.time()
    print("Time taken (before optimizations):", end_time - start_time, "seconds")

    # Measure time taken to load data after implementing optimizations
    start_time = time.time()
    load_data_parallel(data)
    end_time = time.time()
    print("Time taken (after optimizations):", end_time - start_time, "seconds")

# Run the script
if __name__ == '__main__':
    main()
